# 2022/06/24: 学問への扉（音と画像のデジタル処理）
--- 

画像のフィルタリングについての実験をします．  
【発展】と書いてある部分は，余裕があればやってみてください．

## 画像の読み込み
---

前回と同様に，[OpenCV](https://opencv.org)を使って画像を読み込みます．
- [OpenCVの公式ドキュメント（英語）](https://docs.opencv.org/4.3.0/)

> **実験**  
> 1. `# 初期設定` から始まる下のブロックのプログラムをそのまま実行しましょう．ブロックにカーソルを合わせると左上に三角の再生ボタンのようなものが出てくるので，そこをクリックするとプログラムを実行できます．左にチェックマークがついたら実行が完了しています．
> 1. その下の `# 以下を実行すると〜` から始まるブロックのプログラムを実行しましょう．「ファイル選択（Choose Files）」をクリックして好きな画像をアップロードしてください（他人には公開されません）．
> 自分が持っている画像でも良いですし，素材サイト（https://www.pakutaso.com など）からダウンロードした画像でも構いません．


In [ ]:
# 初期設定
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
import copy
import cv2 # opencvをインポート


# 関数をいくつか定義
def load_image():
    """
    画像を読み込む関数

    """
    uploaded_file = files.upload()
    file_name = next(iter(uploaded_file)) # ファイル名取得
    print('file name =', file_name)
    img = cv2.imread(file_name) # 画像読み込み

    return img


def show_image(img, title='', fontsize=20): 
    """
    画像を表示する関数

    入力
    ----------
    img: 配列
        表示したい画像
    title: 文字列
        画像のタイトル（なくてもよい）
    fontsize: 整数
        タイトルのフォントサイズ（デフォルト：20）
    
    """
    
    fig, ax = plt.subplots()
    img_converted = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ax.imshow(img_converted)
    ax.axis('off')
    ax.set_title(title, fontsize=fontsize)
    plt.show()


def add_noise_Gaussian(img, std):
    """
    画像にガウスノイズを付加する

    入力
    ----------
    img: 配列
        ノイズを付加したい画像
    std: 実数
        ノイズの標準偏差（大きいほど画像が劣化する）
    
    """

    rng = np.random.default_rng() # 乱数生成器の作成
    noise = rng.standard_normal(np.shape(img)) * std
    img_noisy = img + noise
    img_noisy[img_noisy>255] = 255
    img_noisy[img_noisy<0] = 0
    img_noisy = img_noisy.astype(np.uint8)

    return img_noisy


def add_noise_impulse(img, p):
    """
    画像にインパルスノイズを付加する

    入力
    ----------
    img: 配列
        ノイズを付加したい画像
    p: 実数, 0-1
        ノイズ付加する画素の割合（大きいほど画像が劣化する）
    
    """

    rng = np.random.default_rng() # 乱数生成器の作成
    random_mat = rng.random(img.shape)
    img_noisy = copy.deepcopy(img)
    img_noisy[random_mat > 1 - p/2] = 255
    img_noisy[random_mat < p/2] = 0
    img_noisy = img_noisy.astype(np.uint8)

    return img_noisy


In [ ]:
# 以下を実行するとファイルをアップロードするフォームが出てくるので
# 「ファイル選択（Choose Files）」から画像ファイルをアップロードする
# （画像サイズが大きいと時間がかかるので，5MB程度までが無難）
img = load_image() # imgという変数に画像の情報が格納される

## ノイズの付加
---

ノイズ除去の実験では，ノイズ除去の手法の性能を確かめるために，ノイズを付加した画像を人工的に作ることがあります．

> **実験**  
> 1. 下の `# ガウスノイズの付加` から始まるブロックのプログラムをそのまま実行してみましょう．
> ガウスノイズを付加する前の画像と付加した後の画像が表示されます．
> 1. `std`という値を変えると，どれくらいの大きさのノイズを付加するかが変わります．
> `std`が大きいほど大きなノイズが付加されます．  
> `std=20`となっているところの数値をいろいろ変えてプログラムを実行してみて，結果がどのようになるか観察しましょう．
> 1. さらに下の `# インパルスノイズの付加` から始まるブロックを実行して，インパルスノイズを付加した画像を表示しましょう．
> 1. `p`（0から1まで）の値を変えると，インパルスノイズによって変化する画素数の割合が変わります．  
> `p=0.2`となっているところの数値をいろいろ変えてプログラムを実行してみて，結果がどのようになるか観察しましょう．

In [ ]:
# ガウスノイズの付加
std = 20  # ノイズの標準偏差
img_noisy_Gaussian = add_noise_Gaussian(img, std)  # ノイズ付加後の画像をimg_noisy_Gaussianという変数に保存

show_image(img, 'original')  # 元の画像を表示
show_image(img_noisy_Gaussian, f'image with Gaussian noise (std={std})')  # ガウスノイズ付加後の画像を表示

In [ ]:
# インパルスノイズの付加
p = 0.2  # ノイズを付加する画素の割合
img_noisy_impulse = add_noise_impulse(img, p)  # ノイズ付加後の画像をimg_noisy_impulseという変数に保存

show_image(img, 'original')  # 元の画像を表示
show_image(img_noisy_impulse, f'image with impulse noise (p={p})')  # インパルスノイズ付加後の画像を表示

## 画像の保存
---

Googleドライブと連携して，画像ファイルを保存しましょう．

> **実験**  
> 1. 下の2つのブロックのプログラムを順番に実行し，ガウスノイズを付加した画像がGoogleドライブに保存されることを確認しましょう．
> 2. 2つ目のブロックで`filename`の部分を他の文字列に変えると，そのファイル名で画像を保存できます．また，`img_noisy_Gaussian`の部分を他の画像を表す変数にすると，その画像を保存できます．  
先ほどとは別のファイル名を好きに決めて，`img_noisy_impulse`の中身を画像ファイルとして保存しましょう．


In [ ]:
from google.colab import drive

# Googleドライブをマウント（自分のGoogleアカウントのGoogleドライブにプログラムからアクセスできるようにする）
# 何か許可を求められたら許可してください
drive.mount('/content/drive')

In [ ]:
# 変数img_noisy_Gaussianに入っている画像をfilename.jpgとしてGoogleドライブの「マイドライブ」に保存
cv2.imwrite('/content/drive/My Drive/filename.jpg', img_noisy_Gaussian)

## 平均化フィルタ

---

フィルタリングを使うと画像をぼけさせることができ，ノイズの影響を抑えられる場合があります．

> **実験**  
> 1. 下のブロックのプログラムをそのまま実行して，平均化フィルタでフィルタリングした結果を表示してみましょう．フィルタリング前とフィルタリング後のPSNR（Peak Signal to Noise Ratio）の値も表示されます（値が大きい方がノイズのないきれいな画像に近い）
> 1. `kernel_size`の値を変えたときの結果の画像やPSNRの値がどのようになるか観察しましょう．
> 1. 上のブロックでガウスノイズの大きさを変えたときの画像を作成し，同様にフィルタリングを行ってみましょう．
> 1. インパルスノイズを含む画像に対して，同様にフィルタリングを行ってみましょう．下のブロックの`img_noisy_Gaussian`をすべて`img_noisy_impulse`に変更する必要があることに注意してください．

In [ ]:
# 平均化フィルタ
kernel_size = 3  # カーネルのサイズ（縦と横の幅）
img_filtered_box = cv2.boxFilter(img_noisy_Gaussian, -1, (kernel_size, kernel_size))  # フィルタリング

show_image(img_noisy_Gaussian, 'noisy image')  # ノイズを含む画像img_noisy_Gaussianを表示
show_image(img_filtered_box, 'filtered image by Box filter')  # フィルタリング後の画像img_filtered_boxを表示

print('PSNR of noisy image', cv2.PSNR(img, img_noisy_Gaussian))  # ノイズを含む画像img_noisy_GaussianのPSNRを表示
print('PSNR of filtered image', cv2.PSNR(img, img_filtered_box))  # フィルタリング後の画像img_filtered_boxのPSNRを表示

##  ガウシアンフィルタ

---

> **実験**  
> 1. 下のブロックのプログラムは，ガウシアンフィルタを`img_noisy_Gaussian`の画像にかけた結果の画像を`img_filtered_Gaussian`という変数に入れるものです．  
> 平均化フィルタの場合の上のプログラムを参考にして，フィルタリング前後の画像とPSNRの値を表示するプログラムを完成させましょう．
> 1. 平均化フィルタのときと同様に，カーネルのサイズ・画像に含まれるノイズの大きさ・ノイズの種類（ガウスノイズ or インパルスノイズ）などを変えたときの結果を観察しましょう．  
> 平均化フィルタを用いた場合と比べて，ガウシアンフィルタを用いた場合は結果の画像やPSNRの値にどのような違いがあるかを見てみましょう．

In [ ]:
# ガウシアンフィルタ
kernel_size = 3  # カーネルのサイズ（奇数）  
img_filtered_Gaussian = cv2.GaussianBlur(img_noisy_Gaussian, (kernel_size, kernel_size), 0)  # フィルタリング


## メディアンフィルタ

---

> **実験**  
> 1. 下のブロックのプログラムは，メディアンフィルタを`img_noisy_Gaussian`の画像にかけた結果の画像を`img_filtered_median`という変数に入れるものです．  
> 平均化フィルタの場合の上のプログラムを参考にして，フィルタリング前後の画像とPSNRの値を表示するプログラムを完成させましょう．
> 1. 平均化フィルタのときと同様に，カーネルのサイズ・画像に含まれるノイズの大きさ・ノイズの種類（ガウスノイズ or インパルスノイズ）などを変えたときの結果を観察しましょう．  
> 平均化フィルタやガウシアンフィルタを用いた場合と比べて，メディアンフィルタを用いた場合は結果の画像やPSNRの値にどのような違いがあるかを見てみましょう．

In [ ]:
# メディアンフィルタ
kernel_size = 3  # カーネルのサイズ（奇数）
img_filtered_median = cv2.medianBlur(img_noisy_Gaussian, kernel_size)  # フィルタリング


## 【発展】カーネルのサイズ・ノイズの大きさ・結果の画質などの関係をグラフ化

---

グラフを使うと，実験結果を視覚的に理解しやすくなります．


> **実験**  
> 1. 下のプログラムをそのまま実行しましょう．`x`という配列の中の数値を横軸，`y`という配列の中の数値を縦軸としてプロットしたグラフが表示されます．
> 1. `x`や`y`の配列の中身を変更して，横軸を`kernel_size`の値，縦軸をフィルタリング後の画像のPSNRにしたグラフを作成してみましょう（フィルタに入力する画像やフィルタは好きなもので固定）．
> 1. 同様に，横軸を入力画像に含まれるノイズの大きさ，縦軸をフィルタリング後の画像のPSNRにしたグラフを作成してみましょう（ノイズの種類・`kernel_size`・フィルタなどは好きなもので固定）．
> 1. その他に自分が観察してみたいものがあれば，横軸と縦軸を適切に設定したグラフを作成して結果を見てみましょう．

In [ ]:
# プロットする値の配列
x = np.array([0, 1, 3, 5, 10])
y = np.array([1, 4, 4, 8, 0])
# 軸のラベル
x_label = 'x'
y_label = 'y'

# プロット
fig, ax = plt.subplots()
ax.plot(x, y)
ax.set_xlabel(x_label, fontsize=16)
ax.set_ylabel(y_label, fontsize=16)
ax.tick_params(labelsize=16)
ax.grid(linestyle=':')
plt.show()

## 【発展】自作のフィルタ

---

自分でカーネルを作ってフィルタリングを行うこともできます．

> **実験**  
> 1. 下のプログラムをそのまま実行しましょう．カーネルとして
> $$
    \frac{1}{9}
    \begin{bmatrix}
        1 & 1 & 1 \\
        1 & 1 & 1 \\
        1 & 1 & 1
    \end{bmatrix}
  $$
> を用いたとき（平均化フィルタ）のフィルタリングの結果が表示されます．
> 1. 自分でカーネルの値やサイズを変更してみて，どのように結果が変わるか見てみましょう．


In [ ]:
# 自作のフィルタ
kernel = np.array([[1, 1, 1],
                   [1, 1, 1],
                   [1, 1, 1]])
kernel_normalized = kernel / np.sum(kernel)  # カーネルを正規化（合計を1にする）
print(kernel_normalized)
img_filtered = cv2.filter2D(img_noisy_Gaussian, -1, kernel_normalized)  # フィルタリング

show_image(img_noisy_Gaussian, 'noisy image')  # ノイズを含む画像を表示
show_image(img_filtered, 'filtered image')  # フィルタリング後の画像を表示

print('PSNR of noisy image', cv2.PSNR(img, img_noisy_Gaussian))
print('PSNR of filtered image', cv2.PSNR(img, img_filtered))

## 【おまけ】 エッジ抽出

---

フィルタリングは，ノイズ除去だけでなく画像の勾配（画素の変化量）やエッジ（物体の境界）を得るためにも使われます．

> **実験**  
> 1. 下のプログラムをそのまま実行しましょう．カーネルとして
> $$
    \begin{bmatrix}
        0 & -1 & 0 \\
        -1 & 4 & -1 \\
        0 & -1 & 0
    \end{bmatrix}
  $$
> を用いたときのフィルタをきれいな画像`img`にかけたときの結果が表示されます．
> 1. 「エッジ抽出　フィルタ」などと検索すると様々なフィルタが出てきます．  
> それらを実際に試してどのような結果になるか見てみましょう．



In [ ]:
# エッジ抽出
kernel = np.array([[0, -1, 0],
                   [-1, 4, -1],
                   [0, -1, 0]])
print(kernel)  # カーネルを表示
img_filtered = cv2.filter2D(img, -1, kernel)  # フィルタリング（エッジ抽出）

show_image(img, 'original')  # 元の画像を表示
show_image(img_filtered, 'filtered image')  # フィルタリング後の画像を表示